In [2]:
import numpy as np
import pandas as pd
import scipy as sc
import re

## Day 4

In [146]:
file1 = open('day4_input', 'r') 
input4_lines = file1.readlines() 

passport_entry_names = ['iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid', 'byr', 'cid']
essential_entries_names = passport_entry_names[:-1]

In [147]:
essential_entries_names

['iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid', 'byr']

In [148]:
passports = []
passport=[]
for line in input4_lines:
    # New passport entry
    if line == '\n':
        passports.append(passport)
        passport = []
    else:
        # Remove newlines
        line1 = line.replace('\n','')
        # Split over spaces
        entry_list = line1.split(' ')
        for entry in entry_list:
            passport.append(entry)
            
passports.append(passport)

In [149]:
paspports_df = pd.DataFrame(columns=passport_entry_names, index=range(len(passports)))
for idx, passport in enumerate(passports):
    for entry in passport:
        key, value = entry.split(':')
        if not key in passport_entry_names:
            print(key)
        paspports_df.loc[idx, key] = value

In [150]:
paspports_df.head()

,iyr,eyr,hgt,hcl,ecl,pid,byr,cid
0,2013,2034,176cm,#fffffd,amb,934693255,2000,89
1,2017,2020,155cm,#b5c3db,grn,#baec97,1939,NaN
2,2023,1972,152cm,z,dne,526669252,1960,NaN
3,2016,2028,73in,#c0946f,#473aaf,565318180,1926,NaN
4,2019,2030,170cm,#62e117,oth,472686027,1940,277


### Part A

In [151]:
passports_missing_essentials_nbr = paspports_df[essential_entries_names].isnull().sum(axis=1)

In [152]:
passports_missing_essentials_nbr = passports_missing_essentials_nbr > 0

In [153]:
passports_missing_essentials_nbr.sum()

25

In [ ]:
len(passports) - passports_invalid.sum()

### Part B

In [156]:
import re
import functools

In [ ]:
help(functools)

In [169]:
# Define regex expression for each entry type.
#s='1945'
#re.fullmatch("[1920-20002]", s)

#byr 1920 <= yr <= 2002
regex_byr=r"^\d{4}$"
#iyr #2010-2020
regex_iyr=r"^\d{4}$"
#eyr 2020-2030
regex_eyr=r"^\d{4}$"
#hgt cm:150-193, in:59-76
regex_hgt=r"^(\d{3,3}cm|\d{2,2}in)$"
#hcl
regex_hcl=r"^#[0-9a-f]{6}$"
#ecl
regex_ecl=r"^(amb|blu|brn|gry|grn|hzl|oth)$"
#pid
regex_pid=r"^\d{9}$"

In [171]:
# decorator: Check whether string fullfils given regex.
def valid_entry_regex(func):
    @functools.wraps(func)
    def wrapper_valid_entry_regex(*args, **kwargs):
        #print(args)
        if args[0] is np.nan:
            return False
        
        if not re.fullmatch(args[1], args[0]) is None:
            return func(*args, **kwargs)
        else:
            return False  
        
    return wrapper_valid_entry_regex

# Input string is 4 digit year within specified range.
@valid_entry_regex
def valid_year(entry, regex, low, high):
    #check if given number is within range
    
    if (int(entry) >= low) & (int(entry) <= high):
        return True
    else:
        return False
    
# Validity of input string is set by regex only.    
@valid_entry_regex
def valid_def(entry, regex):
    return True

# Input sting is value + unit
@valid_entry_regex
def valid_hgt_regex(hgt, regex):
    
    match = re.match(r"(\d+)(cm|in)", hgt, re.I)
    if match:
        length, unit = match.groups()
        length = int(length)
        
        if unit == 'cm':
            if (length >= 150) & (length <= 193):
                return True
        if unit == 'in':
            if (length >= 59) & (length <= 76):
                return True
            
    return False

# Validity checkers for input. Each function requires only a string as input.   
def valid_byr(entry):
    return valid_year(entry, regex_byr, 1920, 2002)

def valid_iyr(entry):
    return valid_year(entry, regex_iyr, 2010, 2020)

def valid_eyr(entry):
    return valid_year(entry, regex_eyr, 2020, 2030)

def valid_hgt(hgt):
    return valid_hgt_regex(hgt, regex_hgt)

def valid_hcl(entry):
    return valid_def(entry, regex_hcl)

def valid_ecl(entry):
    return valid_def(entry, regex_ecl)

def valid_pid(entry):
    return valid_def(entry, regex_pid)


In [172]:
paspports_df['valid_byr'] = paspports_df['byr'].apply(valid_byr)
paspports_df['valid_eyr'] = paspports_df['eyr'].apply(valid_eyr)
paspports_df['valid_iyr'] = paspports_df['iyr'].apply(valid_iyr)
paspports_df['valid_hgt'] = paspports_df['hgt'].apply(valid_hgt)
paspports_df['valid_hcl'] = paspports_df['hcl'].apply(valid_hcl)
paspports_df['valid_ecl'] = paspports_df['ecl'].apply(valid_ecl)
paspports_df['valid_pid'] = paspports_df['pid'].apply(valid_pid)

In [175]:
paspports_df.head()

,iyr,eyr,hgt,hcl,ecl,pid,byr,cid,valid_byr,valid_eyr,valid_iyr,valid_hgt,valid_hcl,valid_ecl,valid_pid
0,2013,2034,176cm,#fffffd,amb,934693255,2000,89,True,False,True,True,True,True,True
1,2017,2020,155cm,#b5c3db,grn,#baec97,1939,NaN,True,True,True,True,True,True,False
2,2023,1972,152cm,z,dne,526669252,1960,NaN,True,False,False,True,False,False,True
3,2016,2028,73in,#c0946f,#473aaf,565318180,1926,NaN,True,True,True,True,True,False,True
4,2019,2030,170cm,#62e117,oth,472686027,1940,277,True,True,True,True,True,True,True


In [185]:
essential_entries_names_validity = ['valid_' + s for s in essential_entries_names]

answer4B = paspports_df[essential_entries_names_validity].all(axis=1).sum()
print('Number of valid passoorts:', answer4B)

Number of valid passoorts: 156
